In [1]:
import os
import rasterio
from rasterio import windows
from rasterio.enums import Resampling
from scipy.ndimage import generic_filter
import pandas as pd
import numpy as np
import re
from datetime import datetime
print("modules imported")

modules imported


In [2]:
dem = r"D:/ASOML/Rockies/features/ASO_CON_dem_albn83_60m.tif"

In [3]:
def min_max_scale(data, min_val=None, max_val=None, feature_range=(0, 1)):
    """Min-Max normalize a NumPy array to a target range."""
    data = data.astype(np.float32)
    mask = np.isnan(data)

    d_min = np.nanmin(data) if min_val is None else min_val
    d_max = np.nanmax(data) if max_val is None else max_val

    # if d_max == d_min:
    #     raise ValueError("Min and max are equal — can't scale.")
    if d_max == d_min:
        return np.full_like(data, feature_range[0], dtype=np.float32)

    a, b = feature_range
    scaled = (data - d_min) / (d_max - d_min)  # to [0, 1]
    scaled = scaled * (b - a) + a              # to [a, b]

    scaled[mask] = np.nan  # preserve NaNs
    return scaled

In [5]:
# loop through all the files
inputFolder = "D:/ASOML/Rockies/features/"
outputFolder = "D:/ASOML/Rockies/features/scaled/"
raster_list = []

# for filename in os.listdir(inputFolder):
# #     if filename.endswith(".tif"):
# print(filename)
input_path = os.path.join(inputFolder, "ASOML_CON_waterbodies_1_0_60_albn83.tif")
output_path = os.path.join(outputFolder, "ASOML_CON_waterbodies_1_0_60_albn83_csl.tif")

with rasterio.open(input_path) as src:
    data = src.read(1).astype(np.float32)
    data[data == src.nodata] = np.nan  

    scaled_data = min_max_scale(data)

    # Save the scaled raster
    profile = src.profile
    profile.update(dtype=rasterio.float32, nodata=np.nan)

    with rasterio.open(output_path, 'w', **profile) as dst:
        scaled_data = np.nan_to_num(scaled_data, nan=profile["nodata"])
        dst.write(scaled_data, 1)
    print(f"{output_path} saved")

D:/ASOML/Rockies/features/scaled/ASOML_CON_waterbodies_1_0_60_albn83_csl.tif saved


In [ ]:
output_raster = r"D:/ASOML/Rockies/features/ASOML_SNM_ProCurv_60_albn83_scl.tif"
input_ras = r"D:/ASOML/Sierras/features/ASOML_SNM_ProCurv_60_albn83.tif"
with rasterio.open(input_ras) as src:
    data = src.read(1, masked=True).filled(np.nan)  # Read first band as numpy array
    profile = src.profile  # Get the metadata for saving later

# Apply min-max scaling
scaled_data = min_max_scale(data, feature_range=(0, 1))

# Optional: Convert NaNs to a NoData value if needed
nodata_value = profile.get('nodata', -1)
scaled_data = np.where(np.isnan(scaled_data), nodata_value, scaled_data)

# Update profile if needed
profile.update(dtype=rasterio.float32, nodata=nodata_value)

# Save scaled raster
with rasterio.open(output_raster, 'w', **profile) as dst:
    dst.write(scaled_data.astype(rasterio.float32), 1)

print(f"Scaled raster saved to: {output_raster}")

In [3]:
def elevation_std_3x3(dem_path, output_path):
    import numpy as np
    import rasterio
    from scipy.ndimage import generic_filter

    with rasterio.open(dem_path) as src:
        dem = src.read(1, masked=True)
        profile = src.profile.copy()

    # Convert to float so np.nan can be used
    dem = dem.astype(np.float32)
    
    # Replace masked values with np.nan
    dem_data = dem.filled(np.nan)

    # Define a std filter that ignores NaNs
    def nanstd_filter(x):
        return np.nanstd(x)

    # Apply 3x3 std filter
    std_data = generic_filter(dem_data, nanstd_filter, size=3)

    # Update profile for float output
    profile.update(dtype='float32', nodata=np.nan)

    with rasterio.open(output_path, 'w', **profile) as dst:
        dst.write(std_data.astype(np.float32), 1)

In [4]:
elevation_std_3x3(dem, r"D:\ASOML\Rockies\features\ASOML_CON_stdElv_60_albn83.tif")

C:\Users\etyrr\anaconda3\envs\aso-dl\lib\site-packages\numpy\lib\nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


In [ ]:
def slope_std_3x3(slope_raster_path, output_path):
    with rasterio.open(slope_raster_path) as src:
        slope = src.read(1, masked=True)
        profile = src.profile.copy()

    # Replace NoData with np.nan for proper std calculation
    slope_data = slope.filled(np.nan)

    # Define function that computes std ignoring NaNs
    def nanstd_filter(x):
        return np.nanstd(x)

    # Apply 3x3 window
    slope_std = generic_filter(
        slope_data,
        function=nanstd_filter,
        size=3,
        mode='nearest'
    )

    # Restore NoData areas
    slope_std = np.where(np.isnan(slope_data), -9999, slope_std)

    # Save to output raster
    profile.update(dtype='float32', nodata=-9999)

    with rasterio.open(output_path, 'w', **profile) as dst:
        dst.write(slope_std.astype('float32'), 1)

In [ ]:
# this is to great the gradient mangintue 
def compute_gradient_magnitude(dem_path, output_path):
    with rasterio.open(dem_path) as src:
        dem = src.read(1, masked=True)
        transform = src.transform
        res_x = transform[0]    # pixel width
        res_y = -transform[4]
        profile = src.profile.copy()

    # Compute partial derivatives using numpy gradient
    dem_data = dem.filled(np.nan)
    dz_dy, dz_dx = np.gradient(dem, res_y, res_x)  # note: y comes first
    grad_mag = np.sqrt(dz_dx**2 + dz_dy**2)
    grad_mag = np.where(np.isnan(dem_data), -9999, grad_mag)

    # Save the result as a new raster
    profile = src.profile.copy()
    profile.update(dtype='float32', nodata=None)

    with rasterio.open(output_path, 'w', **profile) as dst:
        dst.write(grad_mag.astype('float32'), 1)

In [ ]:
slope_std_3x3(r"D:\ASOML\Rockies\features\ASO_CON_slope_albn83_60m.tif", r"D:\ASOML\Rockies\features\ASO_CON_STDslope_albn83_60m.tif")

In [ ]:
compute_gradient_magnitude(dem, r"D:/ASOML/Rockies/features/ASO_CON_gradMag_60_albn83.tif")

In [ ]:
with rasterio.open(dem) as src:
    dem = src.read(1, masked=True)
    profile = src.profile.copy()
    transform = src.transform
    res_x = transform.a
    res_y = -transform.e  # make it positive

# Convert to float32
dem_data = dem.astype('float32')
dem_data = np.where(dem.mask, np.nan, dem_data)

# Compute gradient
dz_dy, dz_dx = np.gradient(dem_data, res_y, res_x)
grad_mag = np.sqrt(dz_dx**2 + dz_dy**2)

# Save the result as a new raster
profile = src.profile.copy()
profile.update(dtype='float32', nodata=None)

with rasterio.open(r"D:/ASOML/Rockies/features/ASO_CON_gradMag_60_albn83.tif", 'w', **profile) as dst:
    dst.write(grad_mag.astype('float32'), 1)